In [1]:
import pandas                        as pd
import numpy                         as np
import math

In [6]:
## Load the row document data
raw_doc=pd.read_csv('./data/eczema-post-relavant.csv', encoding = "iso-8859-1" , index_col=None, header = 0)

In [ ]:
## Define the interested topic number and the threshold for presence of a topic in the documents

ntopics = 400
threshold=0.01

In [2]:
docTopic=pd.read_csv('./data/topics/doc-topics-400-1.txt', encoding = "iso-8859-1" , index_col=None, header = None)

In [3]:
docTopic.head()

,0
0,0\t85:0.132 362:0.016 312:0.016 304:0.016 288:...
1,1\t203:0.067 389:0.056 344:0.045 19:0.045 185:...
2,2\t344:0.067 376:0.034 165:0.034 147:0.034 347...
3,3\t292:0.048 357:0.039 261:0.039 8:0.039 373:0...
4,4\t393:0.062 344:0.05 252:0.038 227:0.038 208:...


In [ ]:
## The the prevalence of a topic in the documents based on the weight and threshold
## Get the maximal weight of a topic and its coresponding document text
## Calculate the maximal weight inverted document frequence

for i in [1]:
    #print(i)
    
    topic=pd.read_csv('./data/topics/topic-keys-400-'+ str(i) + '.txt', encoding = "iso-8859-1" , index_col=None, header = None)
    
    docTopic=pd.read_csv('./data/topics/doc-topics-400-'+ str(i) + '.txt', encoding = "iso-8859-1" , index_col=None, header = None)

    topic_doc_weight = []
    topic_doc_ct= []
    topic_mw_idf=[]
    topic_doc_rel = []
    topic_doc_ls = []
    
    doc_ct = 0

    for j in range(ntopics):
        topic_doc_weight.insert(j,0)
        topic_doc_rel.insert(j,0)
        topic_doc_ct.insert(j,0)
        topic_doc_ls.insert(j,[])

    for row in docTopic.iterrows():
        doc_ct += 1
        items=row[1].str.split()
        doc_id=items[0][0]
        # Get the topic and its weight for each document
        for t_w in items[0][1:]:
            t, w = t_w.split(':')
            # Get the document counts of each topic when weight greater than threshold
            if float(w) > threshold:
                topic_doc_ct[int(t)]=topic_doc_ct[int(t)]+1
                topic_doc_ls[int(t)].append(int(doc_id))

                # Get the maximal weight for each topic
                if float(w) > topic_doc_weight[int(t)]:
                    topic_doc_weight[int(t)]= float(w)
                    topic_doc_rel[int(t)]= int(doc_id)
                
    # Calculat the max weight IDF for each topic 
    # IDF = log (total_doc_ct/topic_doc_ct)
    # mw-idf = max_weight*IDF
    for t in range(ntopics):
        idf=math.log(doc_ct/topic_doc_ct[t])
        mx= topic_doc_weight[t]
        topic_mw_idf.insert(t,idf*mx)
    
    topics_metric = pd.DataFrame({'doc_ct' : topic_doc_ct,
                        'doc_weight' : topic_doc_weight,
                        'mw_idf' : topic_mw_idf, 
                        'docID' : topic_doc_rel},
                         columns=['doc_ct','doc_weight','mw_idf','docID'])
   
    topics_metric=topics_metric.merge(doc, on='docID', how='left', suffixes=('_1', '_2'))

    topics_idf= pd.concat([topic,topics_metric], axis=1)
                                  
    topics2.to_csv('./data/topics/topic-keys-400-IDF-'+ str(i) + '.csv')

## Upload the topic list with the maximal weight IDF as well as the NLP Topic Category

In [107]:
IDF_1=pd.read_csv('./data/topics/topic-keys-400-IDF-1.csv', encoding = "iso-8859-1" , index_col=None, header = 0)

In [114]:
# rename dataframe columns
#IDF_1.columns = ['topic_id_1', 'topic', 'doc_ct', 'doc_weight', 'mw_idf', 'most_rel_docID', 'most_reltext']

In [77]:
NLPTopicsCategorized = pd.read_csv('./data/NLPTopicsCategorized.csv', encoding = "iso-8859-1" , names=['topic','cat'],index_col=None, header = 0)

In [265]:
## Combine the topic list with the category

column_names = ["topic_id_1", "category"]

topic_cat_1 = pd.DataFrame(columns = column_names)

for row in NLPTopicsCategorized.iterrows():
    items=row[1].str.split()
    topic_cat_1=topic_cat_1.append({"topic_id_1":int(items[0][0].split(';')[2]),"category":row[1]['cat']},ignore_index=True)


In [266]:
topic_cat=IDF_1.merge(topic_cat_1, on='topic_id_1', how='inner', suffixes=('_1', '_2'))
topic_cat.to_csv('./data/topics/topic_cat.csv')

In [259]:
topic_cat_1['category']=topic_cat_1['category'].str.lower().str.strip()

In [261]:
#topic_cat_1['category'].unique()

In [262]:
topic_cat_ls={}
topic_cat_ct={}

for category in topic_cat_1['category'].unique():
    topic_cat_ls[category]=[]
    topic_cat_ct[category]=[]
    print(category)
    for row in topic_cat_1[topic_cat_1['category']==category].iterrows():
        items=row[1].str.split()
        topic_number=int(items[0][0])
        for i in topic_doc_ls[topic_number]:
            if i not in topic_cat_ls[category]:
                topic_cat_ls[category].append(i)
    topic_cat_ct[category].append(len(topic_cat_ls[category]))


?
ain
allergic contact dermatitis jewelry
allergic reactions, delayed reactions (food)
allergy shots & allergy testing
aloe vera (treatment)
alternative approaches to eczema treatment
antihistamines
aron regimen
baby eczema
baby eczema products
bacterial infections of skin
bad eczema flare up
bathing
bedding (choice and maintenance)
benefits of chlorinated water, sea water
beverage triggers (coffee, alcohol)
black box warning (protopic?)
body  areas affected
breast eczema
bullying / stigma
burning caused by topical treatments
burning from topical treatment
cause of eczema
cause of flares
cbd treatment
challenges in the work environment
challenges to maintaining skin health
chemicals and irritants in personal care products
chronic eczema
chronic eczema struggle
cleansers
climate and location (eczema)
climate impacts
clothing & covering skin & sun protection
co-morbidiites
coal pine tar treatment
communication / issues with doctor
contact dermatitis
containers for treatments / air travel

In [268]:
pd.DataFrame.from_dict(topic_cat_ct,orient='index').to_csv('./data/topics/topic_cat_ct.csv')